# Introduction

Welcome to the tutorial for ChemCoord (http://chemcoord.readthedocs.org/).

The manipulation of the coordinates is a lot easier, if you can view them on the fly.
So please install a molecule viewer, which opens xyz-files. 
A non complete list includes:
[molcas gv](http://www.molcas.org/GV/),
[avogadro](https://avogadro.cc/),
[vmd](http://www.ks.uiuc.edu/Research/vmd/), and
[pymol](https://www.pymol.org/)

# Cartesian

In [1]:
import chemcoord as cc
import numpy as np
import time

In [2]:
water = cc.Cartesian.read_xyz('water_dimer.xyz', start_index=1)
middle = cc.Cartesian.read_xyz('MIL53_middle.xyz', start_index=1)

Let's have a look at it:

In [3]:
water

,atom,x,y,z
1,O,0.000000,0.0,0.000000
2,H,0.758602,0.0,0.504284
3,H,0.260455,0.0,-0.872893
4,O,3.000000,0.5,0.000000
5,H,3.758602,0.5,0.504284
6,H,3.260455,0.5,-0.872893


It is also possible to open it with an external viewer. I use Molcas ``gv.exe`` so you have to change it accordingly to your program of choice.

In [4]:
water.view(viewer='gv.exe')

To make this setting permament, execute:

In [5]:
cc.settings['defaults']['viewer'] = 'gv.exe'  # replace by your viewer of choice

# Slicing

The slicing operations are the same as for ``pandas.DataFrames``. (http://pandas.pydata.org/pandas-docs/stable/indexing.html)

If the ``'x'`` axis is of particular interest you can slice it out with:

In [6]:
water['x']
# or explicit label based indexing
water.loc[:, 'x']
# or explicit integer based indexing
water.iloc[:, 1]

1    0.000000
2    0.758602
3    0.260455
4    3.000000
5    3.758602
6    3.260455
Name: x, dtype: float64

With boolean slicing it is very easy to  cut all the oxygens away:

In [7]:
water[water['atom'] != 'O'].view()

This can be combined with other selections:

In [8]:
water[(water['atom'] != 'O') & (water['x'] < 1)].view()

### Returned type

The indexing behaves like Indexing and Selecting data in
[Pandas](<http://pandas.pydata.org/pandas-docs/stable/indexing.html>).
You can slice with `Cartesian.loc[key]`, `Cartesian.iloc[keys]`, and `Cartesian[key]`.
The only question is about the return type.
If the information in the columns is enough to draw a molecule,
an instance of the own class (e.g. `Cartesian`)
is returned.
If the information in the columns is not enough to draw a molecule, there 
are two cases to consider:

* A `pandas.Series` instance is returned for one dimensional slices
* A `pandas.DataFrame` instance is returned in all other cases:

        molecule.loc[:, ['atom', 'x', 'y', 'z']] returns a `Cartesian`.

        molecule.loc[:, ['atom', 'x']]`` returns a `pandas.DataFrame`.

        molecule.loc[:, 'atom']`` returns a `pandas.Series`.

# Sideeffects and Method chaining

Two general rules are: 
1. **All functions are sideeffect free unless stated otherwise in the documentation**.
2. **Where possible the methods return an instance of the own class, to allow method chaining**.

Have a look at the unmodified molecule

In [9]:
middle.view()

Chain the methods:

In [10]:
middle.cutsphere(radius=5, preserve_bonds=False).view()

The molecule itself remains unchanged.

In [11]:
middle.view()

# Chemical bonds

One really important method is ``get_bonds()``. 
It returns a connectivity table, which is represented by a dictionary.
Each index points to set of indices, that are connected to it.

In [12]:
water.get_bonds()

{1: {2, 3}, 2: {1}, 3: {1}, 4: {5, 6}, 5: {4}, 6: {4}}

Now the focus switches to another molecule (MIL53_small)

Let's explore the coordinationsphere of the Cr atom with the index 7.

In [13]:
for i in range(3):
    middle.connected_to(2, n_sphere=i).view()
    time.sleep(1)

We can also partition the molecule into subsets of the same chemical environment

# Binary operators

### Mathematical Operations:

Binary operators are supported in the logic of the scipy stack, but you need
python3.x for using the matrix multiplication operator ``@``.

The general rule is that mathematical operations using the binary operators
``+ - * / @`` and the unary operatos ``+ - abs``
are only applied to the ``['x', 'y', 'z']`` columns.

**Addition/Subtraction/Multiplication/Division**:
If you add a scalar to a Cartesian it is added elementwise onto the
``['x', 'y', 'z']`` columns.
If you add a 3-dimensional vector, list, tuple... the first element of this
vector is added elementwise to the ``'x'`` column of the
Cartesian instance and so on.
The last possibility is to add a matrix with
``shape=(len(Cartesian), 3)`` which is again added elementwise.
The same rules are true for subtraction, division and multiplication.

**Matrixmultiplication**:
Only leftsided multiplication with a matrix of ``shape=(n, 3)``,
where ``n`` is a natural number, is supported.
The usual usecase is for example
``np.diag([1, 1, -1]) @ cartesian_instance``
to mirror on the x-y plane.

In [14]:
(water + 3).view()

In [15]:
rot_mat = cc.utilities.algebra_utilities.rotation_matrix
(rot_mat([1, 0, 0], np.radians(90)) @ water).view()
# If you use python2.x the @ operator is not supported then you have to use:
# cc.xyz_functions.dot(rot_mat([1, 0, 0], np.radians(90)), water).view()

### Comparison:

The comparison operators ``==`` and ``!=`` are supported and require molecules indexed in the same way:

In some cases it is better to test for numerical equality $ |a - b| < \epsilon$. This is done using
``allclose`` or ``isclose`` (elementwise)

In [16]:
water == water + 1e-15

,atom,x,y,z
1,True,False,False,False
2,True,False,False,False
3,True,False,False,False
4,True,False,False,False
5,True,False,False,False
6,True,False,False,False


In [17]:
cc.xyz_functions.isclose(water, water + 1e-15)

,atom,x,y,z
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True
5,True,True,True,True
6,True,True,True,True


In [18]:
cc.xyz_functions.allclose(water, water + 1e-15)

True

# Symbolic evaluation

It is possible to use symbolic expressions from sympy.

In [19]:
import sympy
sympy.init_printing()
x = sympy.Symbol('x')

In [20]:
water['x'] = [x + i for i in range(len(water))]

In [21]:
water

,atom,x,y,z
1,O,$x$,0.0,0.000000
2,H,$x + 1$,0.0,0.504284
3,H,$x + 2$,0.0,-0.872893
4,O,$x + 3$,0.5,0.000000
5,H,$x + 4$,0.5,0.504284
6,H,$x + 5$,0.5,-0.872893


In [22]:
water.subs(x, 2)

,atom,x,y,z
1,O,2.0,0.0,0.000000
2,H,3.0,0.0,0.504284
3,H,4.0,0.0,-0.872893
4,O,5.0,0.5,0.000000
5,H,6.0,0.5,0.504284
6,H,7.0,0.5,-0.872893


In [23]:
water.subs(x, 2).view()

# Alignment

# Symmetry

Pointgroup detection is still to come